In [1]:
import pandas as pd
import numpy as np


In [9]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [2]:
df = pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [3]:
df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [4]:
df.sample()

,age,gender,fever,cough,city,has_covid
31,83,Male,103.0,Mild,Kolkata,No


# Columns Analysis
1. Imputation missing value
  - `Fever` Column has missing value

2. Different Encoding
  + `age` is numarical
  + `gender` nominal
  + `fever` numarical
  + `cough` ordinal
  + `city` nominal
  + `has_covid` label (Nominal)


# Split Train Test Data

In [5]:
X = df.drop(columns=['has_covid'])
y = df['has_covid']

In [6]:
X.sample(2)

,age,gender,fever,cough,city
61,81,Female,98.0,Strong,Mumbai
51,11,Female,100.0,Strong,Kolkata


In [7]:
y.sample(2)

,has_covid
33,No
8,No


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


# Kutta Jibon

## Imputation missing value -> Fever

In [10]:
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.transform(X_test[['fever']])

In [11]:
X_train_fever.shape

(80, 1)

## Ordinalencoding -> cough

In [12]:
ordinal_encoder = OrdinalEncoder(categories=[['Mild', 'Strong']])
X_train_cough = ordinal_encoder.fit_transform(X_train[['cough']])
X_test_cough = ordinal_encoder.transform(X_test[['cough']])

In [13]:
X_train_cough.shape

(80, 1)

## OneHotEncoding -> gender,city

In [14]:
ohe =OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])
X_test_gender_city = ohe.transform(X_test[['gender', 'city']])

In [15]:
X_train_gender_city.shape

(80, 4)

## Extract Age Column

In [16]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

## Concatenate all of these column

In [17]:
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis=1)
X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis=1)

In [19]:
X_train_transformed.shape

(80, 7)

# Smart Life

One Class included
```
1. Imputation
2. Ordinal Encoder
3. One Hot Encoder
```

In [20]:
from sklearn.compose import ColumnTransformer

In [21]:
transformer = ColumnTransformer(
    transformers=[
        ('tnf1', SimpleImputer(), ['fever']),
        ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
        ('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city'])
    ],
    remainder='passthrough'
)

In [22]:
transformer.fit_transform(X_train).shape

(80, 7)

In [23]:
transformer.transform(X_test).shape

(20, 7)